## Step 10: 테스트

소프트웨어 개발에 있어 테스트는 필수적이다. 이는 지금 우리가 개발하고자 하는 DeZero에서도 마찬가지이다.
여기서는 간단한 테스트를 진행해보도록 하겠다.

### 10.1 파이썬 단위 테스트

피이썬으로 테스트할 때는 표준 라이브러리에 포함된 unittest를 사용하면 편하다.

```python
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
```

이와 같이 unittest를 import하여 unittest.TestCase를 상속한 SquareTest 클래스를 구현한다. 이때 테스트할 내용은 test로 시작하는 메서드 안에 넣어 작성해야한다.

위 메서드에서는 출력과 기댓값이 같은지 판단하는 self.assertEqual 메서드를 사용했지만, unittest에는 self.assertGreater, self.assertTrue 등 다양한 메서드를 제공하고 있으므로 필요에 따라 사용하면 된다.

<a href ='https://docs.python.org/ko/3/library/unittest.html'>공식문서</a>

unittest는 터미널에서 다음과 같이 실행할 수 있다.

> $ python -m unittest {테스트코드_위치}

또는 단순히 파이썬 파일 끝에 다음을 추가해도 된다.

```python
unittest.main()
```

### 10.2 square 함수의 역전파 테스트

이어서 square 함수의 역전파도 테스트해보겠다.

```python
class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
    
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
```

이를 실행하면 2개의 테스트가 모두 실행되고, 결과를 확인할 수 있다. 이처럼 테스트 코드를 잘 작성해두고 square 함수가 수정될 때마다 실행시켜주면 함수의 상태를 즉각적으로 알 수 있게 된다.

### 10.3 기울기 확인을 이용한 자동 테스트

지금까지는 테스트 작성에 있어 expected 값을 직접 계산하여 넣어주었다. 간단한 계산에서는 이것이 쉽지만, 복잡한 계산이 되면 이를 일일이 계산하기란 어려울 것이다. 따라서 해당 계산을 자동화할 수 있도록 하겠다.

수치 미분으로 구한 결과와 역전파로 구한 결과를 비교하여 그 차이 정도를 파악하는 방법으로, 기울기 확인(gradient checking) 방법이라고 한다. 이 방식은 기댓값을 알지 못해도 입력값만 알고 있으면 되므로 테스트 효율을 높일 수 있다.

```python
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps) 
    x1 = Variable(x.data + eps) 
    y0 = f(x0) # f(x-h)
    y1 = f(x1) # f(x+h)
    return (y1.data - y0.data) / (2*eps)

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
    
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

    def test_gradient_check(self):
        x = Variable(np.random.rand(1)) # 무작위 입력값 생성
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)
```


기울기 확인을 할 test_gradient_check 메서드 안에서 무작위 입력값을 하나 생성한다. 이어서 역전파로 미분값을 구하고, numerical_diff 함수를 사용해서 수치 미분으로 계산을 수행해본다. 이후 두 결과를 비교하여 차이 정도를 확인하는데, 이때 np.allclose라는 함수를 이용한다. 이 함수는 두 값이 가까운지를 판별하는 함수로, 그 정도는 인수 rtol과 atol로 지정할 수 있다. np.allclose(a,b)에서 a, b가 다음 수식을 만족시키면 True를 반환한다.

$$
|a-b| \leq (atol + rtol *|b|)
$$

### 10.4 테스트 정리

테스트 코드는 하나의 폴더 안에서 정리해두는 것이 바람직하다. 따라서 여기서도 test 디렉토리 안에 저장하도록 하겠다.

테스트 파일들은 다음 명령으로 일괄 실행이 가능하다.

> $ python -m unittest discover {directory}

깃허브 저장소에 트래비스 CI 서비스를 연계해두면 코드 푸시와 풀 리퀘스트 병합이 이루어질 때 자동으로 테스트를 수행하고 문제 발생 시 메일 등으로 보고를 받을 수 있다. 이와 같은 구조를 구축해두면 코드의 신뢰성을 유지할 수 있다.